In [3]:
import os
from dotenv import load_dotenv

from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY")
print(f"API Key loaded successfully. {len(API_KEY)} characters.")

API Key loaded successfully. 164 characters.


In [4]:
llm = ChatOpenAI(
    model="gpt-4.1-nano",
    api_key=API_KEY,
    temperature=0.7,
)

In [5]:
# Dummy knowledge base for writing assistance
WRITING_KNOWLEDGE_BASE = {
    "grammar_rules": {
        "subject_verb_agreement": "Ensure subjects and verbs agree in number (singular/plural)",
        "comma_usage": "Use commas to separate items in a series and before coordinating conjunctions",
        "apostrophes": "Use apostrophes for contractions and possessive nouns"
    },
    "writing_styles": {
        "academic": "Formal tone, complex sentences, third person, evidence-based",
        "creative": "Expressive language, varied sentence structure, emotional engagement",
        "business": "Clear, concise, professional tone, action-oriented"
    },
    "essay_structure": {
        "introduction": "Hook, background, thesis statement",
        "body": "Topic sentence, evidence, analysis, transition",
        "conclusion": "Restate thesis, summarize main points, call to action"
    }
}

In [6]:
@tool
def knowledge_base_search(query: str) -> str:
    """
    Search the writing knowledge base for relevant information.
    """
    query = query.lower()
    results = list()

    for category, items in WRITING_KNOWLEDGE_BASE.items():
        if any(keyword in query for keyword in [category, *items.keys()]):
            results.append(f"\n**{category.replace('_', ' ').title()}:**")
            for key, value in items.items():
                if key in query:
                    results.append(f"- **{key.replace('_', ' ').title()}:** {value}")

    if not results:
        return "No relevant information found in the knowledge base."

    return "\n".join(results)

In [9]:
@tool
def text_analyzer(text: str) -> str:
    """
    Analyze the provided text for grammar, style, and structure.`
    :param text:
    :return:
    """
    if not text.strip():
        return "No text provided for analysis."

    sentences = text.split('.')
    words = text.split()
    paragraphs = text.split('\n\n')

    avg_sentence_length = sum(len(sentence.split()) for sentence in sentences) / len(sentences) if sentences else 0

    analysis = f"""
**Text Analysis:**
- **Total Words:** {len(words)}
- **Total Sentences:** {len(sentences)}
- **Total Paragraphs:** {len(paragraphs)}
- **Average Sentence Length:** {avg_sentence_length:.2f} words
- **Grammar Issues:** {len([word for word in words if word.lower() in ['the', 'and', 'is', 'to', 'of']])} common grammar issues found (e.g., overuse of common words).

**Suggestions:**
- Consider varying sentence structure to enhance readability.
- Use more specific vocabulary to improve clarity and engagement.
- Ensure proper paragraph structure with clear topic sentences and transitions.
"""
    return analysis.strip()

In [10]:
# Create the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert AI Writing Assistant. Your role is to help users improve their writing through:

1. Grammar and style guidance
2. Structural suggestions
3. Content development advice
4. Text analysis and feedback

Guidelines:
- Always be encouraging and constructive
- Provide specific, actionable advice
- Use your tools when appropriate to access knowledge or analyze text
- Explain your reasoning clearly
- Adapt your tone to match the user's writing goals

Available tools:
- knowledge_base_search: For grammar rules, writing styles, and structure information
- text_analyzer: For analyzing text metrics and readability

Always strive to provide comprehensive, helpful assistance that improves the user's writing skills."""),
    MessagesPlaceholder(variable_name="chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# Create tools list
tools = [knowledge_base_search, text_analyzer]

# Create the agent
agent = create_openai_functions_agent(llm, tools, prompt)

# Create agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True
)

In [11]:
# Test the assistant
def run_writing_assistant(user_input: str):
    try:
        response = agent_executor.invoke({"input": user_input})
        return response["output"]
    except Exception as e:
        return f"Error: {str(e)}"

# Example usage
if __name__ == "__main__":
    # Test knowledge base search
    print("=== Testing Knowledge Base ===")
    result1 = run_writing_assistant("What are the rules for academic writing style?")
    print(result1)

    print("\n=== Testing Text Analysis ===")
    sample_text = "This is a sample text. It contains multiple sentences. Let's see how it performs in analysis."
    result2 = run_writing_assistant(f"Please analyze this text: {sample_text}")
    print(result2)

    print("\n=== General Writing Help ===")
    result3 = run_writing_assistant("How can I improve the structure of my essay?")
    print(result3)

=== Testing Knowledge Base ===


> Entering new AgentExecutor chain...

Invoking: `knowledge_base_search` with `{'query': 'rules for academic writing style'}`



**Writing Styles:**
- **Academic:** Formal tone, complex sentences, third person, evidence-basedAcademic writing has several key rules designed to ensure clarity, formality, and credibility. Here are some of the most important guidelines:

1. **Use Formal Language:** Avoid slang, colloquialisms, and contractions. Choose precise and professional vocabulary.

2. **Maintain Objectivity:** Present ideas and arguments without personal bias. Use passive voice judiciously to emphasize the information over the author.

3. **Use Third Person:** Generally, avoid first-person pronouns ("I," "we") unless instructed otherwise. Instead, focus on the research or evidence.

4. **Be Clear and Concise:** Express ideas clearly without unnecessary words. Aim for precision and clarity in your sentences.

5. **Structure Your Work Properly:** Follow